# Google Earth Engine Download Example

This notebook demonstrates how you can view, clip and download GIS Images from the Earth Engine Dataset


In this example a Region from the YKDelta was studied and multiple download links were created to download GeoTiff files for this specific region
For future projects I suggest that the Export.image.toDrive is utilized to store these images on the cloud to prevent a large data requirement on the computer
In addition, the MODIS dataset should be used as the bands in this data set will be essential for most of the Hazard predictions

 Import Required Libraries

In [1]:
import ee
import geemap
import pandas as pd
from IPython.display import Image

Authenticate Google Earth Engine

In [2]:
ee.Authenticate()
ee.Initialize()

Enter verification code: 4/1Adeu5BWRpjK9m7iuBosS3xoqBxVTF8tnDGzYhL2u90CfF-NtT-cNG1oo_Hk

Successfully saved authorization token.


# Importing Data

- Elevation, Slope, Aspect, NDVI, Temperature, Soil Moisture and Precipitation download links were produced
- For NDVI, Temperature, and Soil Moisture a yearly average TIFF was downloaded for each year from 2000-2021.

## Elevation

In [3]:
#Alaska boundary
alaska_bound = ee.FeatureCollection('TIGER/2018/States').filter(ee.Filter.eq('STUSPS','AK'))

#create section of Ykdelta
lon = 62.83
lat = -153
poi = ee.Geometry.Point(lon,lat)
poi_reprojected = poi.transform('EPSG:4269')
box = [-165.5,61,-161,64]
roi =  ee.Geometry.Rectangle(box)

In [4]:
#Import Digital Elevation Model
Dataset = ee.Image('USGS/3DEP/10m')
Elevation = Dataset.select('elevation')

#change alaska boundary to proper coordinate system
alaska_bound_reprojected = alaska_bound.geometry().transform('EPSG:4269')

#clip elevation to just alaska
Elevation_Alaska = Elevation.clip(alaska_bound_reprojected)

url = Elevation_Alaska.getThumbUrl({
    'min': 0, 'max': 2500 ,'dimensions':512,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']})

# Display
Image(url=url)

In [16]:
Elevation_Alaska_reprojected = Elevation_Alaska.reproject(crs='WGS84')

#get download link
link = Elevation_Alaska.getDownloadURL({
    'scale' : 130,
    'crs' : 'EPSG:4269',
    'fileFormat': 'GeoTIFF',
    'region': roi,
    'name': 'elevation',
    'filePerBand': False
})
print(link)

url = Elevation_Alaska.getThumbUrl({
    'min': 0, 'max': 2500 ,'region':roi,'dimensions':512,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']})
Image(url=url)

https://earthengine.googleapis.com/v1/projects/earthengine-legacy/thumbnails/26344f6ba50b89c7da4857d046edb07a-c5f1dec535492e023f9a6afed02d1688:getPixels


## Slope

- Slope is extracted from the previous imported Digital Elevation Module

In [6]:
Slope_Alaska = ee.Terrain.slope(Elevation_Alaska)


url = Slope_Alaska.getThumbUrl({
    'min': 0, 'max': 90 ,'dimensions':512,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']})

# Display
Image(url=url)

In [17]:
#get download link

link = Slope_Alaska.getDownloadURL({
    'scale' : 130,
    'crs' : 'EPSG:4269',
    'fileFormat': 'GeoTIFF',
    'region': roi,
    'name': 'slope',
    'filePerBand': False
})
print(link)

url = Slope_Alaska.getThumbUrl({
    'min': 0, 'max': 90 ,'region':roi,'dimensions':512,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']})
Image(url=url)

https://earthengine.googleapis.com/v1/projects/earthengine-legacy/thumbnails/a381fc1d3f10ae0d675fe78b78d0d293-621254dcf92f535c2f719be780e916aa:getPixels


## Aspect
- Aspect is also extracted from the DEM

In [18]:
Aspect_Alaska = ee.Terrain.aspect(Elevation_Alaska)

url = Aspect_Alaska.getThumbUrl({
    'min': 0, 'max': 360 ,'dimensions':512,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']})

# Display
Image(url=url)

In [19]:
#get download link

link = Aspect_Alaska.getDownloadURL({
    'scale' : 130,
    'crs' : 'EPSG:4269',
    'fileFormat': 'GeoTIFF',
    'region': roi,
    'name': 'aspect',
    'filePerBand': False
})
print(link)

url = Aspect_Alaska.getThumbUrl({
    'min': 0, 'max': 360 ,'region':roi,'dimensions':512,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']})
Image(url=url)

https://earthengine.googleapis.com/v1/projects/earthengine-legacy/thumbnails/c0a2d60bb2f5aee06dac0c1fd4bcff08-b970f4dd16237f8d9b4267e3ec4d7ae9:getPixels


## NDVI

In [10]:
#NDVI
Dataset = ee.ImageCollection('MODIS/061/MYD13Q1').filterDate('2018-01-01','2018-02-01')

NDVI = Dataset.select('NDVI')
#NDVI_Alaska = NDVI.clip('alaska_bound_reprojected')
NDVI_Alaska = NDVI.mosaic().clip(alaska_bound)
url = NDVI_Alaska.getThumbUrl({
   'min': -2000, 'max': 10000 ,'region':alaska_bound.geometry(), 'dimensions':2048,
   'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']})
Image(url=url)

In [20]:
for i in range(2000,2022,1):
    Start_Date = str(i) + '-01-01'
    End_Date = str(i) + '-12-31'

    Dataset = ee.ImageCollection('MODIS/061/MYD13Q1').filterDate(Start_Date,End_Date).select('NDVI')
    NDVI_Alaska = NDVI.mosaic().clip(alaska_bound)
    #get download link
    link = NDVI_Alaska.getDownloadURL({
        'scale' : 110,
        'crs' : 'EPSG:4269',
        'fileFormat': 'GeoTIFF',
        'region': roi,
        'name': 'NDVI_' + str(i),
        'filePerBand': False    
    })
    print(Start_Date +": " + link)

url = NDVI_Alaska.getThumbUrl({
    'min': -2000, 'max': 10000 ,'region':roi,'dimensions':512,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']})
Image(url=url)
    

2000-01-01: https://earthengine.googleapis.com/v1/projects/earthengine-legacy/thumbnails/cc25e46d53c7f40934e5d1449f980352-86d60b9a1beffe3b23bc47d4a0c64296:getPixels
2001-01-01: https://earthengine.googleapis.com/v1/projects/earthengine-legacy/thumbnails/ef0b328dcab74ce38812b2cc2f90f94d-9a4292db6e4d32ce79cd176385008767:getPixels
2002-01-01: https://earthengine.googleapis.com/v1/projects/earthengine-legacy/thumbnails/5057b3fed59cc5d2e66a7812771a8c18-911a05f0a61d9f851c9f1080c1e8dd79:getPixels
2003-01-01: https://earthengine.googleapis.com/v1/projects/earthengine-legacy/thumbnails/a90232f9819c54fd0f1621a83165889e-2c5be568ed25f6a00fd3c1373e0ab41f:getPixels
2004-01-01: https://earthengine.googleapis.com/v1/projects/earthengine-legacy/thumbnails/2a41a4da5bbf19e95498e1015d360f9a-41e159bd05bef5f7c2a827814565fce2:getPixels
2005-01-01: https://earthengine.googleapis.com/v1/projects/earthengine-legacy/thumbnails/ef01bb31e987c9e1969257b31829bbfe-aad742ed99da4af6d21177e27d0f9560:getPixels
2006-01-01

## Temperature

- Normally this wouldn't be downloaded from Earth Engine as there are more sufficient data sources; however for proof of concept these values were also used
- This specific temperature dataset recorded the max temperature of each month. An average of this was then taken for each year

In [21]:
for i in range(2000,2022,1):
    Start_Date = str(i) + '-01-01'
    End_Date = str(i) + '-12-31'
    Dataset = ee.ImageCollection('IDAHO_EPSCOR/TERRACLIMATE').filterDate(Start_Date,End_Date).select('tmmx')
    avg_maxTemp = Dataset.mean().clip(alaska_bound)
    link = avg_maxTemp.getDownloadURL({
        'scale' : 180,
        'crs' : 'EPSG:4269',
        'fileFormat': 'GeoTIFF',
        'region': roi,
        'name': 'avgmaxTemp_' + str(i),
        'filePerBand': False     
    })
    print(Start_Date +": " + link)
url = avg_maxTemp.getThumbUrl({
    'min': -670, 'max': 576 ,'region':roi,'dimensions':512,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']})
Image(url=url)

2000-01-01: https://earthengine.googleapis.com/v1/projects/earthengine-legacy/thumbnails/1c96c42267306cc61422ea7edef9e979-0ce1e0d20745b83e4d960db02542cd41:getPixels
2001-01-01: https://earthengine.googleapis.com/v1/projects/earthengine-legacy/thumbnails/ba2c7820398ec355260c2b638eb42320-1849fc75157bf17f8fcefce251bdc785:getPixels
2002-01-01: https://earthengine.googleapis.com/v1/projects/earthengine-legacy/thumbnails/0d1cf2c3639823804cdb07e096a05eaf-648d5d105e180d32b6af43f3c915f498:getPixels
2003-01-01: https://earthengine.googleapis.com/v1/projects/earthengine-legacy/thumbnails/13f75b4b1c274ebd596e7c0384d4aef5-04b2c40bf565c0ac6b6efa1f5a4c8a7d:getPixels
2004-01-01: https://earthengine.googleapis.com/v1/projects/earthengine-legacy/thumbnails/c1087af92dda0cfab572499438dc41e8-af118a6841cb8a59ee1bc83f3f447856:getPixels
2005-01-01: https://earthengine.googleapis.com/v1/projects/earthengine-legacy/thumbnails/ee14834578427803ea0344cbab97ae1a-cdc8f3cab7cf0ebaf9c46c471198db96:getPixels
2006-01-01

## Soil Moisture
- average soil moisture per year

In [22]:
for i in range(2000,2022,1):
    Start_Date = str(i) + '-01-01'
    End_Date = str(i) + '-12-31'
    Dataset = ee.ImageCollection('IDAHO_EPSCOR/TERRACLIMATE').filterDate(Start_Date,End_Date).select('soil')
    soil = Dataset.mean().clip(alaska_bound)
    link = soil.getDownloadURL({
        'scale' : 180,
        'crs' : 'EPSG:4269',
        'fileFormat': 'GeoTIFF',
        'region': roi,
        'name': 'soil_' + str(i),
        'filePerBand': False     
    })
    print(Start_Date +": " + link)
url = soil.getThumbUrl({
    'min': -0, 'max': 9000 ,'region':roi,'dimensions':512,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']})
Image(url=url)

2000-01-01: https://earthengine.googleapis.com/v1/projects/earthengine-legacy/thumbnails/d91c6a1249d7faf06e6c5ae32b015967-b68cc41da7ab83ca5b0218e0edad8815:getPixels
2001-01-01: https://earthengine.googleapis.com/v1/projects/earthengine-legacy/thumbnails/5bda046ebce31342cb508289af2091f1-fbb3763c5d52cbb3537f52b18614ba77:getPixels
2002-01-01: https://earthengine.googleapis.com/v1/projects/earthengine-legacy/thumbnails/d7d2a93575edce91790b9964a91f3e57-57a191886428ce1972a42872ea208a52:getPixels
2003-01-01: https://earthengine.googleapis.com/v1/projects/earthengine-legacy/thumbnails/07186026456e43d2ea1cc5a59f65a5c7-ecf698595b40a0f0916188e66e00a907:getPixels
2004-01-01: https://earthengine.googleapis.com/v1/projects/earthengine-legacy/thumbnails/75a7c0ad6650bc76e2ebc534b876beb6-68cc5e9d59e4bab40fff545802bfb341:getPixels
2005-01-01: https://earthengine.googleapis.com/v1/projects/earthengine-legacy/thumbnails/7f02b766ee2cd6b06159e7e7988feaff-4b80e501a64eafa869ec842dcb74e2d3:getPixels
2006-01-01

## Precipitation
- total precipitation per year

In [23]:
for i in range(2000,2022,1):
    Start_Date = str(i) + '-01-01'
    End_Date = str(i) + '-12-31'
    Dataset = ee.ImageCollection('IDAHO_EPSCOR/TERRACLIMATE').filterDate(Start_Date,End_Date).select('pr')
    precip = Dataset.sum().clip(alaska_bound)
    link = precip.getDownloadURL({
        'scale' : 180,
        'crs' : 'EPSG:4269',
        'fileFormat': 'GeoTIFF',
        'region': roi,
        'name': 'precip_' + str(i),
        'filePerBand': False     
    })
    print(Start_Date +": " + link)
url = precip.getThumbUrl({
    'min': -0, 'max': 9000 ,'region':roi,'dimensions':512,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']})
Image(url=url)

2000-01-01: https://earthengine.googleapis.com/v1/projects/earthengine-legacy/thumbnails/3a74d5c7b7dc6a8922b1e77c4af98cd1-41ba81d807b449d6ef71bce6c4910246:getPixels
2001-01-01: https://earthengine.googleapis.com/v1/projects/earthengine-legacy/thumbnails/7f8db027cd23d783714d01d930a20884-940c88b206c28af39c37da3219985f49:getPixels
2002-01-01: https://earthengine.googleapis.com/v1/projects/earthengine-legacy/thumbnails/95dbe23b52e0cf6fb6bb09c399b9eb05-b1f912094b529274dff3493fa5da240c:getPixels
2003-01-01: https://earthengine.googleapis.com/v1/projects/earthengine-legacy/thumbnails/9935d9fa960141f43c240ac4762e67ed-111c02cc69fb098f665a5d31c83f5a16:getPixels
2004-01-01: https://earthengine.googleapis.com/v1/projects/earthengine-legacy/thumbnails/19ebc6b21c0769b06f90021923046c84-75b095388fe90c1b8df07e2416fb1683:getPixels
2005-01-01: https://earthengine.googleapis.com/v1/projects/earthengine-legacy/thumbnails/0de8589a520923d67450236b5a2dbaf4-dc969e6b23093783add6738e172f0579:getPixels
2006-01-01